In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma, beta, zeta, comb
import math

# ------------------------------------------------------------
# کلاس توزیع نمایی (Exponential)
# پارامتر: λ (نرخ) > 0
# f(x) = λ exp(-λ x), x ≥ 0
# ------------------------------------------------------------
class ExponentialDistribution:
    def __init__(self, rate):
        """
        rate: λ (نرخ)
        """
        if rate <= 0:
            raise ValueError("نرخ (λ) باید مثبت باشد")
        self.rate = rate

    def pdf(self, x):
        x = np.asarray(x)
        # مقدار صفر برای x<0
        x = np.where(x < 0, 0, x)
        return self.rate * np.exp(-self.rate * x)

    def mean(self):
        return 1 / self.rate

    def variance(self):
        return 1 / (self.rate ** 2)

    def plot(self, data=None, x_min=0, x_max=5, num_points=500):
        x = np.linspace(x_min, x_max, num_points)
        y = self.pdf(x)
        plt.plot(x, y, 'c-', lw=2, label=f'Exponential(λ={self.rate})')
        if data is not None:
            data = np.asarray(data)
            plt.plot(data, self.pdf(data), 'ro', label='داده‌ها', markersize=6)
        plt.title('توزیع نمایی')
        plt.xlabel('x')
        plt.ylabel('چگالی احتمال')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# کلاس توزیع برنولی (Bernoulli)
# پارامتر: p (احتمال موفقیت) 0≤p≤1
# P(X=x) = p^x (1-p)^(1-x) برای x∈{0,1}
# ------------------------------------------------------------
class BernoulliDistribution:
    def __init__(self, p):
        if p < 0 or p > 1:
            raise ValueError("p باید در بازه [0,1] باشد")
        self.p = p

    def pmf(self, x):
        x = np.asarray(x)
        # اطمینان از اینکه x فقط 0 یا 1 است
        if np.any((x != 0) & (x != 1)):
            raise ValueError("مقادیر x باید 0 یا 1 باشند")
        return np.where(x == 1, self.p, 1 - self.p)

    def mean(self):
        return self.p

    def variance(self):
        return self.p * (1 - self.p)

    def plot(self, data=None):
        categories = [0, 1]
        probs = [1 - self.p, self.p]
        plt.bar(categories, probs, width=0.4, color='orange', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            # محاسبه فراوانی نسبی داده‌ها
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            # رسم نقاط قرمز روی میله‌ها
            for val, f in zip(unique, freq):
                plt.plot(val, f, 'ro', markersize=10, label='داده‌ها' if val == 0 else '')
        plt.title('توزیع برنولی')
        plt.xlabel('x')
        plt.ylabel('احتمال')
        plt.xticks(categories)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# کلاس توزیع پواسون (Poisson)
# پارامتر: λ (میانگین) > 0
# P(X=k) = e^{-λ} λ^k / k! , k=0,1,2,...
# ------------------------------------------------------------
class PoissonDistribution:
    def __init__(self, lam):
        if lam <= 0:
            raise ValueError("λ باید مثبت باشد")
        self.lam = lam

    def pmf(self, k):
        k = np.asarray(k)
        if np.any(k < 0) or np.any(k != np.floor(k)):
            raise ValueError("مقادیر k باید اعداد صحیح نامنفی باشند")
        # محاسبه با استفاده از فرمول (برای مقادیر بزرگ می‌توان از log استفاده کرد اما اینجا ساده)
        return np.exp(-self.lam) * (self.lam ** k) / np.array([math.factorial(int(ki)) for ki in k])

    def mean(self):
        return self.lam

    def variance(self):
        return self.lam

    def plot(self, data=None, k_max=None):
        if k_max is None:
            k_max = int(self.lam + 5 * np.sqrt(self.lam)) + 2
        k_vals = np.arange(0, k_max + 1)
        probs = self.pmf(k_vals)
        plt.bar(k_vals, probs, width=0.6, color='teal', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            # محاسبه فراوانی نسبی داده‌ها
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            plt.plot(unique, freq, 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع پواسون')
        plt.xlabel('k')
        plt.ylabel('احتمال')
        plt.xticks(k_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# کلاس توزیع چندجمله‌ای (Multinomial)
# پارامترها: n (تعداد آزمایش‌ها)، p (بردار احتمالات هر دسته)
# P(X1=x1,...,Xk=xk) = n!/(x1!...xk!) * p1^x1 ... pk^xk
# ------------------------------------------------------------
class MultinomialDistribution:
    def __init__(self, n, p):
        self.n = int(n)
        if self.n <= 0:
            raise ValueError("n باید عددی مثبت باشد")
        p = np.asarray(p)
        if np.any(p < 0) or not np.isclose(np.sum(p), 1):
            raise ValueError("p باید برداری از احتمالات نامنفی با مجموع 1 باشد")
        self.p = p
        self.k = len(p)

    def pmf(self, x):
        """
        x: بردار تعداد successes در هر دسته (طول k) که مجموع آنها باید n باشد
        """
        x = np.asarray(x)
        if x.shape != (self.k,) and x.ndim != 1:
            raise ValueError(f"x باید برداری به طول {self.k} باشد")
        if np.any(x < 0) or np.any(x != np.floor(x)):
            raise ValueError("مقادیر x باید اعداد صحیح نامنفی باشند")
        if np.sum(x) != self.n:
            raise ValueError(f"مجموع x باید برابر n={self.n} باشد")

        # محاسبه ضریب چندجمله‌ای: n!/(x1! ... xk!)
        # استفاده از تابع گاما برای فاکتوریل: gamma(n+1) = n!
        coef = gamma(self.n + 1) / np.prod([gamma(xi + 1) for xi in x])
        prob = coef * np.prod(self.p ** x)
        return prob

    def mean(self, i=None):
        """میانگین برای دسته i (اگر i داده شود) یا بردار میانگین‌ها"""
        means = self.n * self.p
        if i is not None:
            return means[i]
        return means

    def variance(self, i=None):
        """واریانس برای دسته i (اگر i داده شود) یا بردار واریانس‌ها"""
        variances = self.n * self.p * (1 - self.p)
        if i is not None:
            return variances[i]
        return variances

    def covariance(self, i, j):
        """کوواریانس بین دو دسته i و j (i≠j)"""
        if i == j:
            return self.variance(i)
        return -self.n * self.p[i] * self.p[j]

    def plot(self, data=None):
        """
        رسم نمودار میله‌ای احتمالات نظری (p) و در صورت وجود داده، فراوانی نسبی داده‌ها
        data: بردار تعداد مشاهده شده در هر دسته (طول k) با مجموع n (اختیاری)
        """
        categories = np.arange(1, self.k + 1)  # شماره دسته‌ها (برای نمایش)
        plt.bar(categories, self.p, width=0.6, color='salmon', edgecolor='black', alpha=0.7, label='احتمالات نظری (p)')
        if data is not None:
            data = np.asarray(data)
            if data.shape != (self.k,) or np.sum(data) != self.n:
                raise ValueError(f"داده باید برداری به طول {self.k} با مجموع {self.n} باشد")
            freq = data / self.n  # فراوانی نسبی
            plt.plot(categories, freq, 'ro', label='فراوانی نسبی داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع چندجمله‌ای')
        plt.xlabel('دسته')
        plt.ylabel('احتمال / فراوانی نسبی')
        plt.xticks(categories)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# کلاس توزیع گاما
# ------------------------------------------------------------
class GammaDistribution:
    def __init__(self, shape, scale):
        if shape <= 0 or scale <= 0:
            raise ValueError("پارامترهای shape و scale باید مثبت باشند")
        self.shape = shape
        self.scale = scale

    def pdf(self, x):
        x = np.asarray(x)
        # مقدار صفر برای xهای منفی
        x = np.where(x < 0, 0, x)
        coef = 1 / (gamma(self.shape) * (self.scale ** self.shape))
        return coef * (x ** (self.shape - 1)) * np.exp(-x / self.scale)

    def mean(self):
        return self.shape * self.scale

    def variance(self):
        return self.shape * (self.scale ** 2)

    def plot(self, data=None, x_min=0, x_max=10, num_points=500):
        x = np.linspace(x_min, x_max, num_points)
        y = self.pdf(x)
        plt.plot(x, y, 'b-', lw=2, label=f'Gamma(α={self.shape}, θ={self.scale})')
        if data is not None:
            data = np.asarray(data)
            plt.plot(data, self.pdf(data), 'ro', label='داده‌ها', markersize=6)
        plt.title('توزیع گاما')
        plt.xlabel('x')
        plt.ylabel('چگالی احتمال')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# کلاس توزیع وایبل
# ------------------------------------------------------------
class WeibullDistribution:
    def __init__(self, shape, scale):
        if shape <= 0 or scale <= 0:
            raise ValueError("پارامترهای shape و scale باید مثبت باشند")
        self.shape = shape
        self.scale = scale

    def pdf(self, x):
        x = np.asarray(x)
        x = np.where(x < 0, 0, x)
        coef = self.shape / self.scale
        return coef * (x / self.scale) ** (self.shape - 1) * np.exp(-(x / self.scale) ** self.shape)

    def mean(self):
        from scipy.special import gamma
        return self.scale * gamma(1 + 1/self.shape)

    def variance(self):
        from scipy.special import gamma
        lam = self.scale
        k = self.shape
        return lam**2 * (gamma(1 + 2/k) - (gamma(1 + 1/k))**2)

    def plot(self, data=None, x_min=0, x_max=10, num_points=500):
        x = np.linspace(x_min, x_max, num_points)
        y = self.pdf(x)
        plt.plot(x, y, 'r-', lw=2, label=f'Weibull(k={self.shape}, λ={self.scale})')
        if data is not None:
            data = np.asarray(data)
            plt.plot(data, self.pdf(data), 'ro', label='داده‌ها', markersize=6)
        plt.title('توزیع وایبل')
        plt.xlabel('x')
        plt.ylabel('چگالی احتمال')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# کلاس توزیع بتا
# ------------------------------------------------------------
class BetaDistribution:
    def __init__(self, alpha, beta):
        if alpha <= 0 or beta <= 0:
            raise ValueError("پارامترهای alpha و beta باید مثبت باشند")
        self.alpha = alpha
        self.beta = beta

    def pdf(self, x):
        x = np.asarray(x)
        # خارج از بازه [0,1] مقدار صفر
        x = np.where((x < 0) | (x > 1), 0, x)
        B = beta(self.alpha, self.beta)
        return (x ** (self.alpha - 1)) * ((1 - x) ** (self.beta - 1)) / B

    def mean(self):
        return self.alpha / (self.alpha + self.beta)

    def variance(self):
        a = self.alpha
        b = self.beta
        return (a * b) / ((a + b)**2 * (a + b + 1))

    def plot(self, data=None, num_points=500):
        x = np.linspace(0, 1, num_points)
        y = self.pdf(x)
        plt.plot(x, y, 'g-', lw=2, label=f'Beta(α={self.alpha}, β={self.beta})')
        if data is not None:
            data = np.asarray(data)
            plt.plot(data, self.pdf(data), 'ro', label='داده‌ها', markersize=6)
        plt.title('توزیع بتا')
        plt.xlabel('x')
        plt.ylabel('چگالی احتمال')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# کلاس توزیع هندسی (گسسته)
# ------------------------------------------------------------
class GeometricDistribution:
    def __init__(self, p):
        if p <= 0 or p > 1:
            raise ValueError("p باید در بازه (0,1] باشد")
        self.p = p

    def pmf(self, k):
        k = np.asarray(k)
        # بررسی صحیح بودن مقادیر (اعداد طبیعی مثبت)
        if np.any(k < 1) or np.any(k != np.floor(k)):
            raise ValueError("مقادیر k باید اعداد صحیح مثبت (1,2,3,...) باشند")
        return ((1 - self.p) ** (k - 1)) * self.p

    def mean(self):
        return 1 / self.p

    def variance(self):
        return (1 - self.p) / (self.p ** 2)

    def plot(self, data=None, k_max=10):
        k_vals = np.arange(1, k_max + 1)
        probs = self.pmf(k_vals)
        plt.bar(k_vals, probs, width=0.6, color='purple', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            # برای داده‌های گسسته، نقاط قرمز روی میله‌ها رسم می‌کنیم
            plt.plot(data, self.pmf(data), 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع هندسی')
        plt.xlabel('k (تعداد آزمایش‌ها تا اولین موفقیت)')
        plt.ylabel('احتمال')
        plt.xticks(k_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# تابع دریافت داده از کاربر
# ------------------------------------------------------------
def get_data_from_user(prompt="داده‌ها را وارد کنید (با فاصله جدا کنید): "):
    data_str = input(prompt)
    try:
        data = [float(x) for x in data_str.split()]
        return data
    except ValueError:
        print("ورودی نامعتبر. لطفاً اعداد را با فاصله وارد کنید.")
        return None

# ------------------------------------------------------------
# توزیع زتا (Zeta) - گسسته
# ------------------------------------------------------------
class ZetaDistribution:
    def __init__(self, s):
        if s <= 1:
            raise ValueError("پارامتر s باید بزرگتر از 1 باشد")
        self.s = s
        self.zeta_s = zeta(s)

    def pmf(self, k):
        k = np.asarray(k)
        if np.any(k < 1) or np.any(k != np.floor(k)):
            raise ValueError("مقادیر k باید اعداد صحیح مثبت (1,2,3,...) باشند")
        return (k ** (-self.s)) / self.zeta_s

    def mean(self):
        if self.s <= 2:
            return np.inf
        return zeta(self.s - 1) / self.zeta_s

    def variance(self):
        if self.s <= 3:
            return np.inf
        m = self.mean()
        return zeta(self.s - 2) / self.zeta_s - m**2

    def plot(self, data=None, k_max=None):
        if k_max is None:
            k = 1
            while True:
                if self.pmf(k) < 0.001 and k > 10:
                    k_max = k
                    break
                k += 1
                if k > 100:
                    k_max = 100
                    break
        k_vals = np.arange(1, k_max + 1)
        probs = self.pmf(k_vals)
        plt.bar(k_vals, probs, width=0.6, color='cyan', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            plt.plot(unique, freq, 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع زتا')
        plt.xlabel('k')
        plt.ylabel('احتمال')
        plt.xticks(k_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# توزیع فوق هندسی (Hypergeometric)
# ------------------------------------------------------------
class HypergeometricDistribution:
    def __init__(self, N, K, n):
        if N <= 0 or K < 0 or n < 0 or K > N or n > N:
            raise ValueError("پارامترهای نامعتبر")
        self.N = int(N)
        self.K = int(K)
        self.n = int(n)

    def pmf(self, x):
        x = np.asarray(x)
        lower = max(0, self.n + self.K - self.N)
        upper = min(self.n, self.K)
        if np.any(x < lower) or np.any(x > upper) or np.any(x != np.floor(x)):
            raise ValueError(f"x باید در بازه [{lower}, {upper}] و عدد صحیح باشد")
        return comb(self.K, x) * comb(self.N - self.K, self.n - x) / comb(self.N, self.n)

    def mean(self):
        return self.n * self.K / self.N

    def variance(self):
        return self.n * (self.K / self.N) * (1 - self.K / self.N) * ((self.N - self.n) / (self.N - 1))

    def plot(self, data=None, x_max=None):
        lower = max(0, self.n + self.K - self.N)
        upper = min(self.n, self.K)
        if x_max is None:
            x_max = upper
        x_vals = np.arange(lower, x_max + 1)
        probs = self.pmf(x_vals)
        plt.bar(x_vals, probs, width=0.6, color='magenta', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            plt.plot(unique, freq, 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع فوق هندسی')
        plt.xlabel('x')
        plt.ylabel('احتمال')
        plt.xticks(x_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# توزیع دوجمله‌ای منفی (Negative Binomial) - تعداد شکست‌ها
# ------------------------------------------------------------
class NegativeBinomialDistribution:
    def __init__(self, r, p):
        if r <= 0 or not float(r).is_integer():
            raise ValueError("r باید عدد صحیح مثبت باشد")
        if p <= 0 or p > 1:
            raise ValueError("p باید در بازه (0,1] باشد")
        self.r = int(r)
        self.p = p

    def pmf(self, k):
        k = np.asarray(k)
        if np.any(k < 0) or np.any(k != np.floor(k)):
            raise ValueError("مقادیر k باید اعداد صحیح نامنفی باشند")
        return comb(k + self.r - 1, k) * (self.p ** self.r) * ((1 - self.p) ** k)

    def mean(self):
        return self.r * (1 - self.p) / self.p

    def variance(self):
        return self.r * (1 - self.p) / (self.p ** 2)

    def plot(self, data=None, k_max=None):
        if k_max is None:
            mean_val = self.mean()
            std_val = np.sqrt(self.variance())
            k_max = int(mean_val + 5 * std_val) + 2
        k_vals = np.arange(0, k_max + 1)
        probs = self.pmf(k_vals)
        plt.bar(k_vals, probs, width=0.6, color='lime', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            plt.plot(unique, freq, 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع دوجمله‌ای منفی (تعداد شکست‌ها)')
        plt.xlabel('k')
        plt.ylabel('احتمال')
        plt.xticks(k_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# توزیع یکنواخت پیوسته (Continuous Uniform)
# ------------------------------------------------------------
class UniformContinuous:
    def __init__(self, a, b):
        if a >= b:
            raise ValueError("کران پایین (a) باید کوچکتر از کران بالا (b) باشد")
        self.a = a
        self.b = b

    def pdf(self, x):
        x = np.asarray(x)
        # مقدار صفر برای x خارج از بازه [a,b]
        return np.where((x >= self.a) & (x <= self.b), 1.0 / (self.b - self.a), 0.0)

    def mean(self):
        return (self.a + self.b) / 2

    def variance(self):
        return ((self.b - self.a) ** 2) / 12

    def plot(self, data=None, num_points=500):
        x = np.linspace(self.a - 0.2 * (self.b - self.a), self.b + 0.2 * (self.b - self.a), num_points)
        y = self.pdf(x)
        plt.plot(x, y, 'b-', lw=2, label=f'Uniform Continuous(a={self.a}, b={self.b})')
        if data is not None:
            data = np.asarray(data)
            plt.plot(data, self.pdf(data), 'ro', label='داده‌ها', markersize=6)
        plt.title('توزیع یکنواخت پیوسته')
        plt.xlabel('x')
        plt.ylabel('چگالی احتمال')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()


# ------------------------------------------------------------
# توزیع یکنواخت گسسته (Discrete Uniform)
# ------------------------------------------------------------
class UniformDiscrete:
    def __init__(self, a, b):
        """
        a و b اعداد صحیح (شامل هر دو)
        """
        if a >= b:
            raise ValueError("کران پایین (a) باید کوچکتر از کران بالا (b) باشد")
        self.a = int(a)
        self.b = int(b)
        self.n = self.b - self.a + 1

    def pmf(self, x):
        x = np.asarray(x)
        # بررسی صحیح بودن مقادیر x (باید اعداد صحیح در بازه [a,b] باشند)
        if np.any(x < self.a) or np.any(x > self.b) or np.any(x != np.floor(x)):
            raise ValueError(f"مقادیر x باید اعداد صحیح در بازه [{self.a}, {self.b}] باشند")
        return np.full_like(x, 1.0 / self.n, dtype=float)

    def mean(self):
        return (self.a + self.b) / 2

    def variance(self):
        return ((self.b - self.a + 1) ** 2 - 1) / 12

    def plot(self, data=None):
        x_vals = np.arange(self.a, self.b + 1)
        probs = self.pmf(x_vals)
        plt.bar(x_vals, probs, width=0.8, color='orange', edgecolor='black', alpha=0.7, label='توزیع نظری')
        if data is not None:
            data = np.asarray(data)
            # محاسبه فراوانی نسبی داده‌ها
            unique, counts = np.unique(data, return_counts=True)
            freq = counts / len(data)
            plt.plot(unique, freq, 'ro', label='داده‌ها', markersize=8, linestyle='None')
        plt.title('توزیع یکنواخت گسسته')
        plt.xlabel('x')
        plt.ylabel('احتمال')
        plt.xticks(x_vals)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()

# ------------------------------------------------------------
# تابع کمکی برای دریافت داده از کاربر (اختیاری)
# ------------------------------------------------------------
def get_data_from_user(prompt="داده‌ها را وارد کنید (با فاصله جدا کنید): "):
    data_str = input(prompt)
    try:
        data = [float(x) for x in data_str.split()]
        return data
    except ValueError:
        print("ورودی نامعتبر. لطفاً اعداد را با فاصله وارد کنید.")
        return None